In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *

In [4]:
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [5]:
PATH = "data/dogscats_kaggle/"

In [6]:
sz=224
bs = 64

In [7]:
!ls {PATH}

models	sample_submission.csv  test  test.zip  tmp  train  train.zip  valid


In [8]:
# Uncomment the below if you need to reset your precomputed activations
!rm -rf {PATH}tmp

In [9]:
arch=resnet34
tfms = tfms_from_model(resnet34, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms,bs = bs, test_name = "test")
learn = ConvLearner.pretrained(resnet34,data)
learn.precompute = False
learn.unfreeze()
lr=np.array([1e-4,1e-3,1e-2])
learn.fit(lr, 2, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

epoch:   0, train_loss: 0.064376, val_loss: 0.042147, accuracy: 0.983984
epoch:   1, train_loss: 0.043425, val_loss: 0.029771, accuracy: 0.988281
epoch:   2, train_loss: 0.033683, val_loss: 0.029659, accuracy: 0.987109



In [10]:
learn.save('224_lastlayer')

In [13]:
learn.load('224_lastlayer')

In [11]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy(probs, y)

0.9896

In [12]:
log_preds_test, y_test = learn.TTA(is_test = True)
probs_test = np.mean(np.exp(log_preds_test), 0)

In [13]:
f=data.test_dl.dataset.fnames
f


['test/7596.jpg',
 'test/1677.jpg',
 'test/749.jpg',
 'test/10225.jpg',
 'test/914.jpg',
 'test/4857.jpg',
 'test/3105.jpg',
 'test/8289.jpg',
 'test/1759.jpg',
 'test/4902.jpg',
 'test/1719.jpg',
 'test/5446.jpg',
 'test/5955.jpg',
 'test/4357.jpg',
 'test/1342.jpg',
 'test/3075.jpg',
 'test/5812.jpg',
 'test/9391.jpg',
 'test/2712.jpg',
 'test/8531.jpg',
 'test/6562.jpg',
 'test/9961.jpg',
 'test/7054.jpg',
 'test/787.jpg',
 'test/644.jpg',
 'test/2167.jpg',
 'test/354.jpg',
 'test/9571.jpg',
 'test/5091.jpg',
 'test/12106.jpg',
 'test/182.jpg',
 'test/70.jpg',
 'test/704.jpg',
 'test/1353.jpg',
 'test/11656.jpg',
 'test/11877.jpg',
 'test/10893.jpg',
 'test/2500.jpg',
 'test/6457.jpg',
 'test/8756.jpg',
 'test/10644.jpg',
 'test/4130.jpg',
 'test/2871.jpg',
 'test/12201.jpg',
 'test/569.jpg',
 'test/4107.jpg',
 'test/1897.jpg',
 'test/8866.jpg',
 'test/10089.jpg',
 'test/11534.jpg',
 'test/6792.jpg',
 'test/8362.jpg',
 'test/1285.jpg',
 'test/7886.jpg',
 'test/10392.jpg',
 'test/959

In [14]:
f=np.core.defchararray.replace(f,'.jpg','')
f=np.core.defchararray.replace(f,'test/','')


In [15]:
print(np.shape(f))

(12500,)


In [16]:
print(np.shape(probs_test[:,1]))

(12500,)


In [17]:
df = pd.DataFrame({"id":f,"label":probs_test[:,1]})

In [18]:
SUBM = f'{PATH}sub/'
os.makedirs(SUBM, exist_ok=True)

In [19]:
df.to_csv(f'{SUBM}subm.gz', compression='gzip',index=False)

In [20]:
FileLink(f'{SUBM}subm.gz')

/home/jainrashi2804/fastai/courses/dl1/data/dogscats_kaggle/sub/subm.gz